In [2]:
!wget -nc -P data http://nlp.cs.aueb.gr/software_and_datasets/Enron-Spam/preprocessed/enron1.tar.gz

--2023-03-16 17:54:03--  http://nlp.cs.aueb.gr/software_and_datasets/Enron-Spam/preprocessed/enron1.tar.gz
Resolving nlp.cs.aueb.gr (nlp.cs.aueb.gr)... 195.251.248.252
Connecting to nlp.cs.aueb.gr (nlp.cs.aueb.gr)|195.251.248.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1802573 (1.7M) [application/x-gzip]
Saving to: 'data/enron1.tar.gz'

enron1.tar.gz       100%[===================>]   1.72M   506KB/s    in 3.5s    

2023-03-16 17:54:12 (506 KB/s) - 'data/enron1.tar.gz' saved [1802573/1802573]



In [4]:
!tar -xzf data/enron1.tar.gz -C data

In [6]:
!cat ./data/enron1/ham/0007.1999-12-14.farmer.ham.txt

Subject: mcmullen gas for 11 / 99
jackie ,
since the inlet to 3 river plant is shut in on 10 / 19 / 99 ( the last day of
flow ) :
at what meter is the mcmullen gas being diverted to ?
at what meter is hpl buying the residue gas ? ( this is the gas from teco ,
vastar , vintage , tejones , and swift )
i still see active deals at meter 3405 in path manager for teco , vastar ,
vintage , tejones , and swift
i also see gas scheduled in pops at meter 3404 and 3405 .
please advice . we need to resolve this as soon as possible so settlement
can send out payments .
thanks

In [11]:
import glob, os
emails, labels = [], []
partition = 0
spam_file_path = './data/enron1/spam'
ham_file_path = './data/enron1/ham'

# email data 불러와서 활용하기

for fname in glob.glob(os.path.join(spam_file_path, '*txt')):
    with open(fname, 'r', encoding='ISO-8859-1') as f:
        emails.append(f.read())
        labels.append(1)

for fname in glob.glob(os.path.join(ham_file_path, '*txt')):
    with open(fname, 'r', encoding='ISO-8859-1') as f:
        emails.append(f.read())
        labels.append(0)

In [19]:
print(emails[1])

Subject: want to make more money ?
order confirmation . your order should be shipped by january , via fedex .
your federal express tracking number is % random _ word .
thank you for registering . your userid is :
% random _ word
learn to make a fortune with ebay !
complete turnkey system software - videos - turorials
clck
here if you would not like to receive future mailings .



In [22]:
# 데이터 전처리 - 자연어처리, nltk 패키지
# 숫자와 구두점 표기 제거, 사람이름 제거, 불용어 제거, 표제어 원형 복원 등등
import nltk
nltk.download('names')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package names to /Users/mjcho/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mjcho/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/mjcho/nltk_data...


True

In [23]:
from nltk.corpus import names
all_names = set(names.words())

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [24]:
def letters_only(word):
    return word.isalpha()

def clean_text(doc):
    cleaned_doc = []
    for word in doc.split(' '): # ' '기준으로 문장에서 단어 분리
        word = word.lower() # 모두 소문자로 변경

        if letters_only(word) and word not in all_names and len(word) > 2: 
            # 단어 길이가 2가 넘지 않는 경우, letters_only인 경우, all_in names에 포함되지 않은 경우 lemmatizer에 넣음
            cleaned_doc.append(lemmatizer.lemmatize(word))
    
    return ' '.join(cleaned_doc) #문장으로 다시 만들어서 반환

In [25]:
cleaned_emails = [clean_text(doc) for doc in emails]

In [28]:
cleaned_emails[0]

'what your cam are you looking for your looking for companion for friendship love date just good ole then try our brand new site wa developed and help anyone find what they looking for quick bio form and you the road satisfaction every sense the word matter may out and youll amazed terrific time this and ste the add res you see the line below into your browser come the site www meganbang biz bld acc more www naturalgolden com retract aitken step preemptive shoehorn scaup electrocardiograph movie honeycomb monster war brandywine pietism byrne catatonia encomium lookup intervenor skeleton turn catfish'

In [29]:
# Test, Training dataset split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(cleaned_emails,
                                                    labels,
                                                    test_size=0.33,
                                                    random_state = 0)

In [45]:
# Vectorized data 형태로의 변환
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english', max_features=500)
term_docs_train = cv.fit_transform(X_train) # Fit_transform을 사용해 feature로 사용 가능한 형태로 만든다
term_docs_test = cv.transform(X_test) # Test data에 대해서는 transform을 사용한다

In [46]:
print(term_docs_train[0]) # frequency로 변환하여 벡터화를 진행한다

  (0, 133)	2
  (0, 197)	1
  (0, 9)	1
  (0, 224)	1
  (0, 432)	3


In [47]:
print(term_docs_test[0])

  (0, 9)	1
  (0, 28)	1
  (0, 121)	1
  (0, 133)	1
  (0, 257)	1
  (0, 395)	1
  (0, 409)	1
  (0, 432)	1


In [49]:
feature_names = cv.get_feature_names()
print(feature_names[133]) 
print(feature_names[9])
#이런식으로 feature가 어떤 단어에 해당하는지 확인할 수 있음

enron
actuals


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [50]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=1.0, fit_prior=True)
# 라플라스 smoothing factor를 1.0으로 결정하고, fit_prior는 True면 학습데이터 분포로 Prior설정
clf.fit(term_docs_train, Y_train)
# 모델 학습 완료

MultinomialNB()

In [52]:
predict_prob = clf.predict_proba(term_docs_test)
predict_prob[0:10] 
# 0(정상)에 대한 확률, 1(스팸)에 대한 확률 반환

array([[9.99999999e-001, 1.12649655e-009],
       [1.00000000e+000, 1.95684343e-028],
       [1.00000000e+000, 8.30276362e-016],
       [7.76762132e-011, 1.00000000e+000],
       [1.00000000e+000, 3.67113996e-053],
       [1.00000000e+000, 3.71256966e-143],
       [1.00000000e+000, 7.76210164e-120],
       [1.00000000e+000, 2.80767073e-016],
       [1.00000000e+000, 7.03238079e-045],
       [6.77193629e-001, 3.22806371e-001]])

In [54]:
prediction = clf.predict(term_docs_test)
prediction[1:10]

array([0, 0, 1, 0, 0, 0, 0, 0, 0])

In [61]:
accuracy = clf.score(term_docs_test, Y_test)
print(f'The accuracy using MultinomialNB is : {accuracy:.4f}%')

The accuracy using MultinomialNB is : 0.8984%


In [69]:
import sklearn.metrics

In [74]:
print(sklearn.metrics.classification_report(Y_test, prediction))

              precision    recall  f1-score   support

           0       0.93      0.92      0.93      1222
           1       0.81      0.84      0.83       490

    accuracy                           0.90      1712
   macro avg       0.87      0.88      0.88      1712
weighted avg       0.90      0.90      0.90      1712

